In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-customer-churn-prediction-dataset/Churn_Modelling.csv


In [2]:
dados = pd.read_csv("/kaggle/input/bank-customer-churn-prediction-dataset/Churn_Modelling.csv")
dados

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from keras.layers import Dense, Dropout

2024-07-12 22:38:31.549068: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 22:38:31.549197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 22:38:31.703386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
dados.shape

(10000, 14)

In [5]:
dados.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#pd.unique(dados["Geography"])
pd.Series(dados["Geography"]).value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
dados.groupby(["Exited"]).size()

Exited
0    7963
1    2037
dtype: int64

In [8]:
previsores = dados.iloc[:,3:13].values
classe = dados.iloc[:,13].values
previsores

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [9]:
labelencoder = LabelEncoder()
previsores[:,1] = labelencoder.fit_transform(previsores[:,1])
previsores[:,2] = labelencoder.fit_transform(previsores[:,2])
previsores

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [10]:
x_treino,x_teste, y_treino, y_teste = train_test_split(previsores, classe, test_size = 0.3)
x_treino.shape

(7000, 10)

In [11]:
x_treino = x_treino.astype("float32")
x_teste = x_teste.astype("float32")
x_teste

array([[8.3300000e+02, 0.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.8224777e+05],
       [6.8400000e+02, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 8.0593492e+04],
       [6.3700000e+02, 2.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 2.6057080e+04],
       ...,
       [8.5000000e+02, 0.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.1288770e+04],
       [6.2400000e+02, 2.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1160590e+05],
       [4.4400000e+02, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.4383577e+05]], dtype=float32)

In [12]:
scaler = StandardScaler()
x_treino = scaler.fit_transform(x_treino)
x_teste = scaler.fit_transform(x_teste)
x_teste

array([[ 1.9371611 , -0.9133632 ,  0.89664304, ...,  0.6447962 ,
         0.95501095,  1.4279338 ],
       [ 0.37830365,  0.29799384, -1.115271  , ...,  0.6447962 ,
         0.95501095, -0.33008552],
       [-0.11341646,  1.5093509 , -1.115271  , ..., -1.5508776 ,
        -1.0471084 , -1.2732437 ],
       ...,
       [ 2.1150172 , -0.9133632 ,  0.89664304, ...,  0.6447962 ,
        -1.0471084 , -1.1827663 ],
       [-0.24942414,  1.5093509 , -1.115271  , ...,  0.6447962 ,
        -1.0471084 ,  0.20624618],
       [-2.1326077 , -0.9133632 , -1.115271  , ...,  0.6447962 ,
        -1.0471084 ,  0.7636328 ]], dtype=float32)

In [13]:
rede = Sequential()
rede.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 10))
rede.add(Dense(units = 5,kernel_initializer = "uniform", activation = "relu"))
rede.add(Dense(units = 5, kernel_initializer="uniform",activation ="relu"))
rede.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
rede.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
rede.fit(x_treino, y_treino, epochs = 200, validation_data = (x_teste, y_teste))

Epoch 1/200


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7775 - loss: 0.6592 - val_accuracy: 0.8030 - val_loss: 0.4514
Epoch 2/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7914 - loss: 0.4517 - val_accuracy: 0.8030 - val_loss: 0.4371
Epoch 3/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8020 - loss: 0.4260 - val_accuracy: 0.8030 - val_loss: 0.4334
Epoch 4/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7902 - loss: 0.4363 - val_accuracy: 0.8030 - val_loss: 0.4317
Epoch 5/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8002 - loss: 0.4249 - val_accuracy: 0.8030 - val_loss: 0.4299
Epoch 6/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7997 - loss: 0.4254 - val_accuracy: 0.8030 - val_loss: 0.4277
Epoch 7/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7978 - loss: 0.4289 - val_accuracy: 0.8030 - val_loss: 0.4257
Epoch 8/200
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7932 - loss: 0.4365 - val_accuracy: 0.8193

In [14]:
rede.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 6)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            35 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 413 (1.62 KB)

 Trainable params: 137 (548.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 276 (1.08 KB)

In [15]:
previsoes = rede.predict(x_teste)
previsoes

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[0.0989695 ],
       [0.67039734],
       [0.0623406 ],
       ...,
       [0.00390493],
       [0.12325098],
       [0.32466474]], dtype=float32)

In [16]:
previsoes = (previsoes > 0.5)
previsoes

array([[False],
       [ True],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [17]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [18]:
matriz = confusion_matrix(previsoes, y_teste)
matriz

array([[2312,  389],
       [  97,  202]])

In [19]:
taxaacerto = accuracy_score(previsoes, y_teste)
taxaacerto

0.838